# Generating command line code for Twint

We have a spreadsheet of MPs' Twitter accounts - and we want to scrape tweets 'at' those accounts. 

To do that we use the tool Twint, inside Docker. 

Docker uses a command line interface, so we need to write some command line code that specifies the searches we want to scrape Twitter for.

We need to run multiple searches because searching for tweets at all 587 accounts would create a file too big to work with.

In [2]:
#import the pandas library for pulling in data
import pandas as pd

## Import the Twitter names

We have a Google spreadsheet with MP names and Twitter accounts. We've published that to the web as a CSV so we can pull it in here.

In [18]:
#store the CSV url
csvurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQsqzb5L_Y10X7ZJ-r4lMWtZUOA0Dvdt3N64F99X3wsy4_XnbHq-Wj2E2sia10o67HznmDdbcyDpRvM/pub?gid=14619868&single=true&output=csv"
#we now have more MP accounts
csvurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTUxP42PNbJg8koPUcfwOvyBxAs_eI8Q9PqqCxA7hPrPLk16c7tmxd3gmnTsSdvHUZnSSGYUe8zI7bp/pub?gid=1596341902&single=true&output=csv"
#fetch the CSV with MP usernames
mpdf = pd.read_csv(csvurl)
#show the first few rows
mpdf.head()

,count,mp_name,mp_surname,on_twitter,account_name,party,constituency,gender,Notes
0,1,Diane Abbott,Abbott,yes,@HackneyAbbott,Labour,Hackney North and Stoke Newington,F,NaN
1,2,Debbie Abrahams,Abrahams,yes,@Debbie_abrahams,Labour,Oldham East and Saddleworth,F,NaN
2,3,Nigel Adams,Adams,yes,@nadams,Conservative,Selby and Ainsty,M,NaN
3,4,Bim Afolami,Afolami,yes,@BimAfolami,Conservative,Hitchin and Harpenden,M,NaN
4,5,Adam Afriyie,Afriyie,yes,@AdamAfriyie,Conservative,Windsor,M,NaN


In [19]:
mpdf.tail()

,count,mp_name,mp_surname,on_twitter,account_name,party,constituency,gender,Notes
595,596,Louie French,French,yes,@louie_french,Conservative,Old Bexley and Sidcup,M,Elected 2021
596,597,Kim Leadbeater,Leadbeater,yes,@kimleadbeater,Labour,Batley and Spen,F,Elected 2021
597,598,Paulette Hamilton,Hamilton,yes,@paulettehamilto,Labour,"Birmingham, Erdington",F,Elected 2022
598,599,Anna Firth,Firth,yes,@anna_firth,Conservative,Southend West,M,Elected 2022
599,600,Tracy Brabin,Brabin,yes,@TracyBrabin,Labour,Batley and Spen,F,resigned 2021


## Create a string for the command

A command to scrape a Twitter search on Twint looks like this

`twint -s '"@ThangamMP" OR "@TanDhesi"'  --since "2022-01-01" -o tweets.tsv --csv`

In this particular example we are conducting a search for tweets at either of the named accounts, since the start of 2022, and saving the results in a comma separated values (.csv) file. The file is named 'tweets.tsv'. 

Why does it end with .tsv rather than .csv? This is a bit of a hack: basically Twint uses tabs to separate columns so that extension ensures that when we import it into Google Sheets or Excel, it will separate each column using tabs rather than commas. 

We need to take each MP's Twitter username and create a string out of all of those.

In [20]:
#use the .join() method - see https://www.programiz.com/python-programming/methods/string/join
joinedstring = '" OR "'.join(mpdf['account_name'])
longstring = "'"+'"'+joinedstring+'"'+"'"
longstring

'\'"@HackneyAbbott" OR "@Debbie_abrahams" OR "@nadams" OR "@BimAfolami" OR "@AdamAfriyie" OR "@twocitiesnickie" OR "@peter_aldous" OR "@rushanaraali" OR "@TahirAliMP" OR "@lucyallan" OR "@DrRosena" OR "@MikeAmesburyMP" OR "@Stuart4WolvesSW" OR "@PutneyFleur" OR "@StuartAndrew" OR "@Caroline_Ansell" OR "@ToniaAntoniazzi" OR "@JonAshworth" OR "@AthertonNWales" OR "@GarethBaconMP" OR "@richardbaconmp" OR "@KemiBadenoch" OR "@Shaun4WBW" OR "@Siobhan_Baillie" OR "@duncancbaker" OR "@SteveBakerHW" OR "@hbaldwin" OR "@SteveBarclay" OR "@HannahB4LiviMP" OR "@PaulaBarkerMP" OR "@baynes_simon" OR "@OrfhlaithBegley" OR "@ApsanaBegumMP" OR "@AaronBell4NUL" OR "@hilarybennmp" OR "@ScottBentonMP" OR "@JakeBerry" OR "@bhatti_saqib" OR "@MhairiBlack" OR "@Ianblackford_MP" OR "@KirstySNP" OR "@BobBlackman" OR "@_OliviaBlake" OR "@PaulBlomfieldMP" OR "@CrispinBlunt" OR "@PeterBoneUK" OR "@StevenBonnarSNP" OR "@PBottomleyMP" OR "@AndrewBowie_MP" OR "@BBradley_Mans" OR "@BenPBradshaw" OR "@SirGrahamBrady"

In [21]:
#it's a LONG string
len(longstring)

11460

In [22]:
#check the end
longstring[-50:]

'ulettehamilto" OR "@anna_firth" OR "@TracyBrabin"\''

### Create a shorter string for 20 MPs

That's going to create too many results, so we need to break it down into batches - say, 20 MPs at a time. 

In [23]:
#how many usernames?
len(mpdf['account_name'])

600

We actually had 599 here, but got it to a round 600 by adding the only MP who was elected in 2019 and then subsequently resigned, Tracy Brabin (3 other MPs died). 

In [24]:
#just join the first 20
joinedstring = '" OR "'.join(mpdf['account_name'][0:20])
longstring = "'"+'"'+joinedstring+'"'+"'"
longstring

'\'"@HackneyAbbott" OR "@Debbie_abrahams" OR "@nadams" OR "@BimAfolami" OR "@AdamAfriyie" OR "@twocitiesnickie" OR "@peter_aldous" OR "@rushanaraali" OR "@TahirAliMP" OR "@lucyallan" OR "@DrRosena" OR "@MikeAmesburyMP" OR "@Stuart4WolvesSW" OR "@PutneyFleur" OR "@StuartAndrew" OR "@Caroline_Ansell" OR "@ToniaAntoniazzi" OR "@JonAshworth" OR "@AthertonNWales" OR "@GarethBaconMP"\''

In [25]:
#create an empty list to store our strings
listofstrings = []

#loop through a range of numbers - 20-100 to begin with
for i in range(20,100,20):
  print(i)
  #join the 20 names up to (but not including) that position
  joinedstring = '" OR "'.join(mpdf['account_name'][i-20:i])
  longstring = "'"+'"'+joinedstring+'"'+"'"
  print(longstring)
  #add to the list
  listofstrings.append(longstring)

print(listofstrings)

20
'"@HackneyAbbott" OR "@Debbie_abrahams" OR "@nadams" OR "@BimAfolami" OR "@AdamAfriyie" OR "@twocitiesnickie" OR "@peter_aldous" OR "@rushanaraali" OR "@TahirAliMP" OR "@lucyallan" OR "@DrRosena" OR "@MikeAmesburyMP" OR "@Stuart4WolvesSW" OR "@PutneyFleur" OR "@StuartAndrew" OR "@Caroline_Ansell" OR "@ToniaAntoniazzi" OR "@JonAshworth" OR "@AthertonNWales" OR "@GarethBaconMP"'
40
'"@richardbaconmp" OR "@KemiBadenoch" OR "@Shaun4WBW" OR "@Siobhan_Baillie" OR "@duncancbaker" OR "@SteveBakerHW" OR "@hbaldwin" OR "@SteveBarclay" OR "@HannahB4LiviMP" OR "@PaulaBarkerMP" OR "@baynes_simon" OR "@OrfhlaithBegley" OR "@ApsanaBegumMP" OR "@AaronBell4NUL" OR "@hilarybennmp" OR "@ScottBentonMP" OR "@JakeBerry" OR "@bhatti_saqib" OR "@MhairiBlack" OR "@Ianblackford_MP"'
60
'"@KirstySNP" OR "@BobBlackman" OR "@_OliviaBlake" OR "@PaulBlomfieldMP" OR "@CrispinBlunt" OR "@PeterBoneUK" OR "@StevenBonnarSNP" OR "@PBottomleyMP" OR "@AndrewBowie_MP" OR "@BBradley_Mans" OR "@BenPBradshaw" OR "@SirGrahamB

### Running on all names now

That works - and we check that the lists run on from each other based on the spreadsheet too. 

In [27]:
#create an empty list to store our strings
listofstrings = []

#loop through a range of numbers - all now
for i in range(20,601,20):
  print(i)
  #join the 20 names up to (but not including) that position
  joinedstring = '" OR "'.join(mpdf['account_name'][i-20:i])
  longstring = "'"+'"'+joinedstring+'"'+"'"
  print(longstring)
  #add to the list
  listofstrings.append(longstring)

print(listofstrings)

20
'"@HackneyAbbott" OR "@Debbie_abrahams" OR "@nadams" OR "@BimAfolami" OR "@AdamAfriyie" OR "@twocitiesnickie" OR "@peter_aldous" OR "@rushanaraali" OR "@TahirAliMP" OR "@lucyallan" OR "@DrRosena" OR "@MikeAmesburyMP" OR "@Stuart4WolvesSW" OR "@PutneyFleur" OR "@StuartAndrew" OR "@Caroline_Ansell" OR "@ToniaAntoniazzi" OR "@JonAshworth" OR "@AthertonNWales" OR "@GarethBaconMP"'
40
'"@richardbaconmp" OR "@KemiBadenoch" OR "@Shaun4WBW" OR "@Siobhan_Baillie" OR "@duncancbaker" OR "@SteveBakerHW" OR "@hbaldwin" OR "@SteveBarclay" OR "@HannahB4LiviMP" OR "@PaulaBarkerMP" OR "@baynes_simon" OR "@OrfhlaithBegley" OR "@ApsanaBegumMP" OR "@AaronBell4NUL" OR "@hilarybennmp" OR "@ScottBentonMP" OR "@JakeBerry" OR "@bhatti_saqib" OR "@MhairiBlack" OR "@Ianblackford_MP"'
60
'"@KirstySNP" OR "@BobBlackman" OR "@_OliviaBlake" OR "@PaulBlomfieldMP" OR "@CrispinBlunt" OR "@PeterBoneUK" OR "@StevenBonnarSNP" OR "@PBottomleyMP" OR "@AndrewBowie_MP" OR "@BBradley_Mans" OR "@BenPBradshaw" OR "@SirGrahamB

### Add the last few

An earlier approach (before we found a dozen MPs whose accounts were missing) left a stray handful of MPs that didn't fit into the 20-step approach we adopted, so a last bit of code can be used to grab them too.

In [ ]:
#join the last few - replace this index with whichever position you want to start from
joinedstring = '" OR "'.join(mpdf['account_name'][580:])
longstring = "'"+'"'+joinedstring+'"'+"'"
print(longstring)
#add to the list
listofstrings.append(longstring)


'"@PeteWishart" OR "@mikejwood" OR "@William_Wragg" OR "@YasinForBedford" OR "@JacobYoungMP" OR "@nadhimzahawi" OR "@DanielZeichner"'


### Adding in the front and end of each

We now have the middle parts of our command - the list of names with 'OR' between - but we also need the front and end bits in this:

`twint -s '"@ThangamMP" OR "@TanDhesi"' --since "2022-01-01" -o tweets.tsv --csv`

In other words, the bits that are the same each time before and after the list of names.

We will need to change the name of the file, too. We could simply take an index (e.g. 1.tsv) or we could take the first and last names in the string.

In [28]:
#based on https://github.com/twintproject/twint/wiki/Basic-usage
#set the start date
startdate = "2022-02-10"
#set the end date
untildate = "2022-02-28"

In [38]:
#create a list to store our commands
commandlist = []

#loop through a series of numbers from 0 to the number of items in that list
for i in range(0,len(listofstrings)):
  #fetch the string at that position
  currentstring = listofstrings[i]
  #create a string which combines that string with some other commands, and specifies a tsv name using the index
  #this version includes the untildate - but that's only useful if the start date is within the last 9 days
  #fullcommand = 'twint -s '+currentstring+' --since "'+startdate+'" --until "'+untildate+'" -o '+str(i)+'.tsv --csv'
  fullcommand = 'twint -s '+currentstring+' --since "'+startdate+'" -o '+str(i)+'.tsv --csv'
  print(fullcommand)
  commandlist.append(fullcommand) 

print(len(commandlist))

twint -s '"@HackneyAbbott" OR "@Debbie_abrahams" OR "@nadams" OR "@BimAfolami" OR "@AdamAfriyie" OR "@twocitiesnickie" OR "@peter_aldous" OR "@rushanaraali" OR "@TahirAliMP" OR "@lucyallan" OR "@DrRosena" OR "@MikeAmesburyMP" OR "@Stuart4WolvesSW" OR "@PutneyFleur" OR "@StuartAndrew" OR "@Caroline_Ansell" OR "@ToniaAntoniazzi" OR "@JonAshworth" OR "@AthertonNWales" OR "@GarethBaconMP"' --since "2022-02-10" -o 0.tsv --csv
twint -s '"@richardbaconmp" OR "@KemiBadenoch" OR "@Shaun4WBW" OR "@Siobhan_Baillie" OR "@duncancbaker" OR "@SteveBakerHW" OR "@hbaldwin" OR "@SteveBarclay" OR "@HannahB4LiviMP" OR "@PaulaBarkerMP" OR "@baynes_simon" OR "@OrfhlaithBegley" OR "@ApsanaBegumMP" OR "@AaronBell4NUL" OR "@hilarybennmp" OR "@ScottBentonMP" OR "@JakeBerry" OR "@bhatti_saqib" OR "@MhairiBlack" OR "@Ianblackford_MP"' --since "2022-02-10" -o 1.tsv --csv
twint -s '"@KirstySNP" OR "@BobBlackman" OR "@_OliviaBlake" OR "@PaulBlomfieldMP" OR "@CrispinBlunt" OR "@PeterBoneUK" OR "@StevenBonnarSNP" OR "

### Create a command which combines them all

To run multiple commands at once in command line (Mac/Linux) you [separate each with a semi-colon](https://stackoverflow.com/questions/8055371/how-do-i-run-two-commands-in-one-line-in-windows-cmd). We just need to join all our commands using a semi-colon and we have a single command we can run. 

In [39]:

#join them all with a semi-colon, so they will all run in command line
' ; '.join(commandlist)

'twint -s \'"@HackneyAbbott" OR "@Debbie_abrahams" OR "@nadams" OR "@BimAfolami" OR "@AdamAfriyie" OR "@twocitiesnickie" OR "@peter_aldous" OR "@rushanaraali" OR "@TahirAliMP" OR "@lucyallan" OR "@DrRosena" OR "@MikeAmesburyMP" OR "@Stuart4WolvesSW" OR "@PutneyFleur" OR "@StuartAndrew" OR "@Caroline_Ansell" OR "@ToniaAntoniazzi" OR "@JonAshworth" OR "@AthertonNWales" OR "@GarethBaconMP"\' --since "2022-02-10" -o 0.tsv --csv ; twint -s \'"@richardbaconmp" OR "@KemiBadenoch" OR "@Shaun4WBW" OR "@Siobhan_Baillie" OR "@duncancbaker" OR "@SteveBakerHW" OR "@hbaldwin" OR "@SteveBarclay" OR "@HannahB4LiviMP" OR "@PaulaBarkerMP" OR "@baynes_simon" OR "@OrfhlaithBegley" OR "@ApsanaBegumMP" OR "@AaronBell4NUL" OR "@hilarybennmp" OR "@ScottBentonMP" OR "@JakeBerry" OR "@bhatti_saqib" OR "@MhairiBlack" OR "@Ianblackford_MP"\' --since "2022-02-10" -o 1.tsv --csv ; twint -s \'"@KirstySNP" OR "@BobBlackman" OR "@_OliviaBlake" OR "@PaulBlomfieldMP" OR "@CrispinBlunt" OR "@PeterBoneUK" OR "@StevenBonna

## Store this in a text file

This is still such a long string that it needs to be stored in a dedicated text file.

In [41]:
#store the string in a variable
stringwithsemicolons =  ' ; '.join(commandlist)
#create a writable ("x") file 
f = open("command.txt", "x") 
#write the variable into that
f.write(stringwithsemicolons)
#close the 'open' object
f.close()

## Store it in a shell file

In fact we could store it in a .sh file which means we could run a command directly from it. 

In [42]:
#store the string in a variable
stringwithsemicolons =  ' ; '.join(commandlist)
#create a writable ("x") file 
f = open("searchmptweets.sh", "x") 
#write the variable into that
f.write(stringwithsemicolons)
#close the 'open' object
f.close()

## Write a command to move the resulting TSV files

Once the script has run, we'll have 30 .tsv files in Docker which we'll need to move out. Now we need another command.

A command to move a single file looks like this:

`docker cp 4d51e4be8baf:root/ortweetsCow_Day.tsv /Users/paul/Downloads/twint-master/ortweetsCow_Day.tsv`

Some code to generate that might look like this...

In [34]:
#store each part of the command
firstpart = "docker cp 4d51e4be8baf:root/"
secondpart = "/Users/paul/Downloads/twint-master/"
filename = "ortweetsCow_Day.tsv"
#combine them, with a space in the middle
print(firstpart+filename+" "+secondpart+filename)

docker cp 4d51e4be8baf:root/ortweetsCow_Day.tsv /Users/paul/Downloads/twint-master/ortweetsCow_Day.tsv


### Generate list of file names

Now this is where it's quite helpful that we named our tsv files numerically: to generate the names of those files we just need to generate a range of numbers, and add '.tsv' to them. 

We will also need to convert the numbers to strings, to avoid an error when we try to combine the two.

In [35]:
#create empty list to store the file names
filelist = []

#loop through the numbers 0 to 29 (it won't include the 30)
for i in range(0,30):
  #create a filename by converting the number to a string and adding .tsv to the end
  filename = str(i)+".tsv"
  print(filename)
  #add it to the list
  filelist.append(filename)

print(filelist)

0.tsv
1.tsv
2.tsv
3.tsv
4.tsv
5.tsv
6.tsv
7.tsv
8.tsv
9.tsv
10.tsv
11.tsv
12.tsv
13.tsv
14.tsv
15.tsv
16.tsv
17.tsv
18.tsv
19.tsv
20.tsv
21.tsv
22.tsv
23.tsv
24.tsv
25.tsv
26.tsv
27.tsv
28.tsv
29.tsv
['0.tsv', '1.tsv', '2.tsv', '3.tsv', '4.tsv', '5.tsv', '6.tsv', '7.tsv', '8.tsv', '9.tsv', '10.tsv', '11.tsv', '12.tsv', '13.tsv', '14.tsv', '15.tsv', '16.tsv', '17.tsv', '18.tsv', '19.tsv', '20.tsv', '21.tsv', '22.tsv', '23.tsv', '24.tsv', '25.tsv', '26.tsv', '27.tsv', '28.tsv', '29.tsv']


### Expand that to generate the whole command

In fact, we can extend this further to create not just the filename but the whole command.

In [36]:
#store each part of the command
firstpart = "docker cp 4d51e4be8baf:root/"
secondpart = "/Users/paul/Downloads/twint-master/"


#create empty list to store the commands
commandlist = []

#loop through the numbers 0 to 29 (it won't include the 30)
for i in range(0,30):
  #create a filename by converting the number to a string and adding .tsv to the end
  filename = str(i)+".tsv"
  #print(filename)
  #combine them, with a space in the middle
  cpcommand = firstpart+filename+" "+secondpart+filename
  print(cpcommand)
  #add it to the list
  commandlist.append(cpcommand)

print(commandlist)

docker cp 4d51e4be8baf:root/0.tsv /Users/paul/Downloads/twint-master/0.tsv
docker cp 4d51e4be8baf:root/1.tsv /Users/paul/Downloads/twint-master/1.tsv
docker cp 4d51e4be8baf:root/2.tsv /Users/paul/Downloads/twint-master/2.tsv
docker cp 4d51e4be8baf:root/3.tsv /Users/paul/Downloads/twint-master/3.tsv
docker cp 4d51e4be8baf:root/4.tsv /Users/paul/Downloads/twint-master/4.tsv
docker cp 4d51e4be8baf:root/5.tsv /Users/paul/Downloads/twint-master/5.tsv
docker cp 4d51e4be8baf:root/6.tsv /Users/paul/Downloads/twint-master/6.tsv
docker cp 4d51e4be8baf:root/7.tsv /Users/paul/Downloads/twint-master/7.tsv
docker cp 4d51e4be8baf:root/8.tsv /Users/paul/Downloads/twint-master/8.tsv
docker cp 4d51e4be8baf:root/9.tsv /Users/paul/Downloads/twint-master/9.tsv
docker cp 4d51e4be8baf:root/10.tsv /Users/paul/Downloads/twint-master/10.tsv
docker cp 4d51e4be8baf:root/11.tsv /Users/paul/Downloads/twint-master/11.tsv
docker cp 4d51e4be8baf:root/12.tsv /Users/paul/Downloads/twint-master/12.tsv
docker cp 4d51e4be8

### Join the commands

Now we can join them just as we did with the twint commands.

In [37]:
#join them all with a semi-colon, so they will all run in command line
' ; '.join(commandlist)

'docker cp 4d51e4be8baf:root/0.tsv /Users/paul/Downloads/twint-master/0.tsv ; docker cp 4d51e4be8baf:root/1.tsv /Users/paul/Downloads/twint-master/1.tsv ; docker cp 4d51e4be8baf:root/2.tsv /Users/paul/Downloads/twint-master/2.tsv ; docker cp 4d51e4be8baf:root/3.tsv /Users/paul/Downloads/twint-master/3.tsv ; docker cp 4d51e4be8baf:root/4.tsv /Users/paul/Downloads/twint-master/4.tsv ; docker cp 4d51e4be8baf:root/5.tsv /Users/paul/Downloads/twint-master/5.tsv ; docker cp 4d51e4be8baf:root/6.tsv /Users/paul/Downloads/twint-master/6.tsv ; docker cp 4d51e4be8baf:root/7.tsv /Users/paul/Downloads/twint-master/7.tsv ; docker cp 4d51e4be8baf:root/8.tsv /Users/paul/Downloads/twint-master/8.tsv ; docker cp 4d51e4be8baf:root/9.tsv /Users/paul/Downloads/twint-master/9.tsv ; docker cp 4d51e4be8baf:root/10.tsv /Users/paul/Downloads/twint-master/10.tsv ; docker cp 4d51e4be8baf:root/11.tsv /Users/paul/Downloads/twint-master/11.tsv ; docker cp 4d51e4be8baf:root/12.tsv /Users/paul/Downloads/twint-master/1

## Write a command to combine the resulting TSV files

We might want to combine all 30 files into one. For that we can use `cat` which simply combines any files ending in .tsv (you need to ensure that there are no other tsv files in the same folder that you don't want included). In command line use `cd` to navigate to the folder containing the tsv files and then type this command:

`cat *.tsv > allfiles.tsv`

## Repeat this for new accounts

After running the code above we discovered a number of MPs not listed who did in fact have Twitter accounts. 

Below we import those specific accounts in order to generate a command for those.

(In the end this isn't useful because it won't go far enough back to cover the same period as the other MPs - so we just run the original code for all the MPs including these ones)

In [4]:
#store the CSV URL
newaccounts = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTUxP42PNbJg8koPUcfwOvyBxAs_eI8Q9PqqCxA7hPrPLk16c7tmxd3gmnTsSdvHUZnSSGYUe8zI7bp/pub?gid=1790941103&single=true&output=csv"
#fetch the CSV 
mpdf = pd.read_csv(newaccounts)
#show the first few rows
mpdf.head()

,count,mp_name,mp_surname,on_twitter,account_name,party,constituency,gender
0,298,Adam Holloway,Holloway,yes,@AdamHollowayMP,Conservative,Gravesham,M
1,166,Allan Dorans,Dorans,no,@allandoransMP,Scottish National Party,"Ayr, Carrick and Cumnock",M
2,135,Angela Crawley,Crawley,no,@AngelaCrawley30,Scottish National Party,Lanark and Hamilton East,F
3,634,Bill Wiggin,Wiggin,yes,@BillWigginMP,Conservative,North Herefordshire,M
4,571,Bob Stewart,Stewart,yes,@BobStewart_BUK,Conservative,Beckenham,M


In [5]:
#use the .join() method - see https://www.programiz.com/python-programming/methods/string/join
joinedstring = '" OR "'.join(mpdf['account_name'])
longstring = "'"+'"'+joinedstring+'"'+"'"
longstring

'\'"@AdamHollowayMP" OR "@allandoransMP" OR "@AngelaCrawley30" OR "@BillWigginMP" OR "@BobStewart_BUK" OR "@DrDanPoulter" OR "@GarethJohnsonMP" OR "@JohnHayesRAPAMP" OR "@Mark_Logan_MP" OR "@NadineDorries" OR "@PBottomleyMP" OR "@UK_FoRBEnvoy"\''

In [7]:
#based on https://github.com/twintproject/twint/wiki/Basic-usage
#set the start date
startdate = "2022-02-10"
#set the end date
untildate = "2022-02-28"
#create a string which combines that string with some other commands, and specifies a tsv name using the index
fullcommand = 'twint -s '+longstring+' --since "'+startdate+'" --until "'+untildate+'" -o '+str(30)+'.tsv --csv'
print(fullcommand)

twint -s '"@AdamHollowayMP" OR "@allandoransMP" OR "@AngelaCrawley30" OR "@BillWigginMP" OR "@BobStewart_BUK" OR "@DrDanPoulter" OR "@GarethJohnsonMP" OR "@JohnHayesRAPAMP" OR "@Mark_Logan_MP" OR "@NadineDorries" OR "@PBottomleyMP" OR "@UK_FoRBEnvoy"' --since "2022-02-10" --until "2022-02-28" -o 30.tsv --csv
